#### En sta segunda parte se trabaja con 4 archivos csv; cada uno representa a una plataforma: Amazon, Disney, Hulu, Netflix. En los archivos se encuentran las películas de cada plataforma e información referente a dichas películas. 

In [97]:
import pandas as pd
import numpy as np

Se cargan los archivos como data frames y se organizan en un diccionario con título 'plataformas'

In [98]:
plataformas = {}
plataformas['amazon'] = pd.read_csv('amazon_prime_titles.csv')
plataformas['disney'] = pd.read_csv('disney_plus_titles.csv')
plataformas['hulu'] = pd.read_csv('hulu_titles.csv')
plataformas['netflix'] = pd.read_csv('netflix_titles.csv')

##### Lo siguiente es sólo un capricho para poder visualizar los data types y valores faltantes en cada columna de todos los data frames en el diccionario al mismo tiempo

Se crea la función extract_info() para obtener la información que queremos

In [99]:
def extract_info(df):
    info_data = []
    for column in df.columns:
        non_null_count = df[column].count()
        dtype = df[column].dtype
        info_data.append((column, non_null_count, dtype))

    return info_data

Se utiliza la función con cada data frame del diccionario y se guarda la información en una lista (plataformas_info)

In [100]:
plataformas_info = []

for platform, df in plataformas.items():
    extracted_info = extract_info(df)
    plataformas_info.append(extracted_info)

Se crea un nuevo data frame con la información extraída

In [101]:
summary_columns = ['Column', 'Non-Null Count', 'Dtype']

summary_df = pd.DataFrame(columns=['Platform'] + summary_columns)

for platform, info_data in zip(plataformas.keys(), plataformas_info):
    temp_df = pd.DataFrame(info_data, columns=summary_columns)
    temp_df['Platform'] = platform
    summary_df = summary_df.append(temp_df, ignore_index=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8500\3716153612.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(temp_df, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8500\3716153612.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(temp_df, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8500\3716153612.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(temp_df, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8500\3716153612.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df 

Se utiliza la función pivot_table para que la información se muestre una columna al lado de otra

In [102]:
summary_pivot = summary_df.pivot_table(index='Column', columns='Platform', values=['Non-Null Count', 'Dtype'], aggfunc='first')


In [103]:
summary_pivot

Dtype                          Non-Null Count               \
Platform      amazon  disney     hulu netflix         amazon disney  hulu   
Column                                                                      
cast          object  object  float64  object           8435   1260     0   
country       object  object   object  object            672   1231  1620   
date_added    object  object   object  object            155   1447  3045   
description   object  object   object  object           9668   1450  3069   
director      object  object   object  object           7586    977     3   
duration      object  object   object  object           9668   1450  2594   
listed_in     object  object   object  object           9668   1450  3073   
rating        object  object   object  object           9331   1447  2553   
release_year   int64   int64    int64   int64           9668   1450  3073   
show_id       object  object   object  object           9668   1450  3073   
title         object  object   object  object           9668   1450  3073   
type          object  object   object  object           9668   1450  3073   

                      
Platform     netflix  
Column                
cast            7982  
country         7976  
date_added      8797  
description     8807  
director        6173  
duration        8804  
listed_in       8807  
rating          8803  
release_year    8807  
show_id         8807  
title           8807  
type            8807

Como primera observación todas las plataformas en todas sus columnas tienen el mismo data type, excepto hulu['cast']

In [104]:
plataformas['hulu']['cast'] = plataformas['hulu']['cast'].astype('object')

Como segunda observación, hulu['cast'] no tiene datos, quizá por lo mismo tenía data type float. Todos los data frames tienen valores faltantes; unos más que otros. Los únicas columnas con datos completos para todas las plataformas son 'type', 'title', 'show_id', 'release_year'

Verifiquemos que las columnas de todos los data frames tengan los nombres iguales (eventualmente buscamos concaternalos)

In [105]:
if (plataformas['amazon'].columns.equals(plataformas['disney'].columns) and
    plataformas['amazon'].columns.equals(plataformas['hulu'].columns) and
    plataformas['amazon'].columns.equals(plataformas['netflix'].columns)):
    print("Todos los data frames tienes los mismos nombres de columnas")
else:
    print("No tienen todos los mismos nombres de columnas")

Todos los data frames tienes los mismos nombres de columnas


Si observamos el 'show_id' (el identificador para la película o serie) que tienen los data frames de las plataformas para su primer registro, podemos ver que es el mismo. Sin embargo cada uno se refiere a una película o serie distinta. 

In [106]:
for platform in plataformas.keys():
    print(f"{platform}     show_id-index_0: {plataformas[platform]['show_id'].iloc[0]}     Title: {plataformas[platform]['title'].iloc[0]}")

amazon     show_id-index_0: s1     Title: The Grand Seduction
disney     show_id-index_0: s1     Title: Duck the Halls: A Mickey Mouse Christmas Special
hulu     show_id-index_0: s1     Title: Ricky Velez: Here's Everything
netflix     show_id-index_0: s1     Title: Dick Johnson Is Dead


Vamos a anteponerle al show_id la primera letra de cada plataforma antes de concatenarlos, para así evitar duplicidad de show_id e igualarlos a los identificadores de las películas del archivo reviews.csv

In [107]:
for platform in plataformas.keys():
    
    # Primero limpiamos de posibles espacios blancos todas las columnas
    plataformas[platform] = plataformas[platform].applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Añadimos la primera letra del nombre de la plataforma a los valores del la columna show_id
    plataformas[platform]['show_id'] = platform[0] + plataformas[platform]['show_id']

Verificamos

In [108]:
for platform in plataformas.keys():
    print(f"{platform}     show_id-index_0: {plataformas[platform]['show_id'].iloc[0]}     Title: {plataformas[platform]['title'].iloc[0]}")

amazon     show_id-index_0: as1     Title: The Grand Seduction
disney     show_id-index_0: ds1     Title: Duck the Halls: A Mickey Mouse Christmas Special
hulu     show_id-index_0: hs1     Title: Ricky Velez: Here's Everything
netflix     show_id-index_0: ns1     Title: Dick Johnson Is Dead


Realizamos la concatenación de los data frames de las plataformas en un nuevo data frame que llamaremos 'plata' para simplificar. 

In [109]:
lista_platas = []
for platform in plataformas.keys():
    lista_platas.append(plataformas[platform])

plata = pd.concat(lista_platas)

In [110]:
plata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.3+ MB


La columna 'rating' en realidad es la clasificación parental: g, pg, pg13, R, etc. Por lo que le cambiaremos el nombre a 'classification'

In [111]:
plata.rename(columns={'rating': 'classification'}, inplace=True)

Se nos pide reemplazar los valores faltantes de la columna 'classification' con 'G'

In [112]:
plata['classification'] = plata['classification'].fillna('G')

revisemos la columna 'classification'

In [113]:
plata['classification'].unique()

array(['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min',
       '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min',
       '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '28 mi

Hay valores en la columna 'classification' que son de la columna 'duration'. Vamos a removerlos y, si en la misma fila el campo de la columna 'duration' no tiene valor, le insertaremos el valor removido. A los campos de los que removamos valores de la columna 'classification' les insertaremos la leyenda 'UNRATED'

In [114]:
plata = plata.reset_index(drop=True)
# creamos columnas temporales
plata['temp_classification'] = ''
plata['temp_duration'] = ''

# extraemos la data de la columna classification 
plata.loc[plata['classification'].str.contains('min|Season'), 'temp_classification'] = plata['classification']
plata.loc[plata['classification'].str.contains('min|Season'), 'classification'] = 'UNRATED'

# extramos la data de duration de las filas correspondientes
plata.loc[plata['temp_classification'] != '', 'temp_duration'] = plata['duration']
plata.loc[(plata['temp_classification'] != '') & (plata['duration'] != ''), 'duration'] = plata['temp_classification']

# eliminamos columnas temporales si no tienen información 
if plata['temp_classification'].isnull().all():
    plata.drop('temp_classification', axis=1, inplace=True)
if plata['temp_duration'].isnull().all():
    plata.drop('temp_duration', axis=1, inplace=True)

Revisemos que los campos de la columna 'duration' que se reemplazaron por el valor que tenía el correspondiente campo en 'classification' no tuvieran valor

In [115]:
plata['temp_duration'].unique()

array(['', nan], dtype=object)

Podemos eliminar nuestras columnas temporales

In [116]:
plata.drop('temp_classification', axis=1, inplace=True)
plata.drop('temp_duration', axis=1, inplace=True)

Le damos formato de fecha a la columna 'date_added'

In [117]:
plata['date_added'] = pd.to_datetime(plata['date_added'].str.strip(), format='%B %d, %Y')
plata['date_added'] = plata['date_added'].dt.strftime('%Y-%m-%d')

Separamos los valores de la columna 'duration' en dos nuevas columnas: 'duration_type', 'duration_int'

In [118]:
# Separamos los números y las letras en columnas diferentes
plata[['duration_int', 'duration_type']] = plata['duration'].str.extract(r'(\d+)\s*(\D+)', expand=True)

# Insertamos las nuevas columnas justo después de la columna "duration"
duration_idx = plata.columns.get_loc('duration')
plata['dur_int'] = ''
plata['dur_type'] = ''
plata.insert(duration_idx + 1, 'duration_int', plata.pop('duration_int'))
plata.insert(duration_idx + 2, 'duration_type', plata.pop('duration_type'))
plata.drop(['duration'], axis=1, inplace=True)

Eliminamos columnas temporales 'dur_int', 'dur_type'

In [119]:
plata.drop(['dur_int', 'dur_type'], axis=1, inplace=True)

Convertimos las columnas con campos de texto a minúsculas

In [120]:
for column in plata.columns:
    if column not in ['show_id', 'date_added', 'release_year', 'duration_int', 'classification']:
        plata[column] = plata[column].apply(lambda x: x.lower() if isinstance(x, str) else x)

Reemplazamos 'seasons' por 'season' para sólo tener dos unidades de medida en duration_type: min, season

In [121]:
plata['duration_type'] = plata['duration_type'].str.replace('seasons', 'season')

Unificamos clasificaciones en la columna 'classification'

In [122]:
# Creamos diccionario con los valores que se reemplazaran y sus reemplazos
replacement_dict = {
    'TV-Y': 'ALL',
    'TV-Y7': '7+',
    'TV-PG': 'PG',
    'TV-14': '13+',
    'TV-NR': 'NR',
    'TV-G': 'G',
    'PG-13': '13+',
    'TV-MA': '18+',
    'NC-17': '18+',
    'UNRATED': 'NR',
    '16': '16+',
    'AGES_16_': '16+',
    'AGES_18_': '18+',
    'ALL_AGES': 'ALL',
    'NOT_RATE': 'NR',
    'TV-Y7-FV': '7+',
    'NOT RATED': 'NR',
    'UR': 'NR'
}

# Reemplazamos los valores de la columna 'classification' utilizando el diccionario replacement_dict
plata['classification'] = plata['classification'].replace(replacement_dict)

Revisamos nuestras clasificaciones finales

In [123]:
plata['classification'].unique()

array(['G', '13+', 'ALL', '18+', 'R', '7+', 'NR', '16+', 'PG'],
      dtype=object)

Exportamos el data frame plata como archivo csv con nombre 'plata.csv'. Aquí termina la segunda_parte

In [124]:
plata.to_csv('plata.csv', index=False)